In [1]:
import polars as pl
import torch
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import vocab
from collections import Counter
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn.utils.rnn import pad_sequence


JP_TRAIN_FILE_PATH = "./kftt-data-1.0/data/orig/kyoto-train.ja"
EN_TRAIN_FILE_PATH = "./kftt-data-1.0/data/orig/kyoto-train.en"

tokenizer_src = get_tokenizer('spacy', language='ja_core_news_sm')
tokenizer_tgt = get_tokenizer('spacy', language='en_core_web_sm')

with open(JP_TRAIN_FILE_PATH, "r", encoding="utf-8")as f:
    train_jp_list = f.readlines()
    train_jp_list = [jp.strip("\n") for jp in train_jp_list]

with open(EN_TRAIN_FILE_PATH, "r", encoding="utf-8")as f:
    train_en_list = f.readlines()
    train_en_list = [en.strip("\n") for en in train_en_list]


In [2]:
class datasets(Dataset):
    def __init__(self, text, label):
        self.jp_datas = text
        self.en_datas = label

    def __len__(self):
        return len (self.jp_datas)

    def __getitem__(self, index):
        jp = self.jp_datas[index]
        en = self.en_datas[index]
        return jp,en

class DataLoaderCreater:

    def __init__(self, src_tokenizer, tgt_tokenizer):
        self.src_tokenizer = src_tokenizer
        self.tgt_tokenizer = tgt_tokenizer

    def build_vocab(self, texts, tokenizer):
        counter = Counter()
        for text in texts:
            counter.update(tokenizer(text))
        specials = ['<unk>', '<pad>', '<start>', '<end>']
        v = vocab(counter, specials=specials, min_freq=1)
        v.set_default_index(v['<unk>'])
        return v

    def convert_text_to_indexes(self, text, vocab, tokenizer):
        return [vocab['<start>']] + [
            vocab[token] if token in vocab else vocab['<unk>'] for token in tokenizer(text.strip("\n"))
        ] + [vocab['<end>']]

    def create_dataloader(self, jp_list, en_list, collate_fn):
        vocab_src = self.build_vocab(jp_list, tokenizer_src)
        vocab_tgt = self.build_vocab(en_list, tokenizer_tgt)
        self.vocab_src_itos = vocab_src.get_itos()
        self.vocab_tgt_itos = vocab_tgt.get_itos()
        self.vocab_src_stoi = vocab_src.get_stoi()
        self.vocab_tgt_stoi = vocab_tgt.get_stoi()
        self.vocab_size_src = len(self.vocab_src_stoi)
        self.vocab_size_tgt = len(self.vocab_tgt_stoi)

        src_data = [torch.tensor(self.convert_text_to_indexes(jp_data, self.vocab_src_stoi, self.src_tokenizer)) for jp_data in jp_list]
        tgt_data = [torch.tensor(self.convert_text_to_indexes(en_data, self.vocab_tgt_stoi, self.tgt_tokenizer)) for en_data in en_list]
        dataset = datasets(src_data, tgt_data)

        dataloader = DataLoader(dataset, batch_size=64, collate_fn=collate_fn)

        return dataloader

PADDING_ID = 1

def collate_fn(batch):
    src_batch = [src for src,tgt in batch]
    tgt_batch = [tgt for src,tgt in batch]

    src_batch = pad_sequence(src_batch, padding_value=PADDING_ID)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PADDING_ID)

dataloader_creater = DataLoaderCreater(tokenizer_src, tokenizer_tgt)
train_dataloader = dataloader_creater.create_dataloader(jp_list=train_jp_list, en_list=train_en_list, collate_fn=collate_fn)
vocab_size_src = dataloader_creater.vocab_size_src
vocab_size_tgt = dataloader_creater.vocab_size_tgt

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import math
# EncodingとEmbeddingの違い
# 学習を行わないのがPositional Encoding
# 学習を行うのがPositional Embedding

class PositionalEncoding():
    def __init__(self, embedding_dim, len_sequence):
        self.embedding_dim = embedding_dim
        self.len_sequence = len_sequence

    def get_sin(self,i,k):
        return torch.sin(i/(10000)**(k/self.len_sequence))

    def get_cos(self,i,k):
        return torch.cos(i/(10000)**(k/self.len_sequence))

    def get_positional_vector(self):
        pe = torch.zeros(self.len_sequence, self.embedding_dim)
        for pos in range(self.len_sequence):
            for i in range(0, int(self.embedding_dim/2)):
                pe[pos, 2*i] = self.get_sin(pos, i)
                pe[pos, 2*i+1] = self.get_cos(pos, i)
        return pe


# Transformerモデルの定義
class TransformerModel(nn.Module):
    def __init__(self, vocab_size_src, vocab_size_tgt, embedding_dim, num_heads, num_layers, device,  dropout=0.1):
        super().__init__()
        self.device = device
        # Positional Encoderを加算する必要あり
        self.embedding_src = nn.Embedding(vocab_size_src, embedding_dim)
        self.embedding_tgt = nn.Embedding(vocab_size_tgt, embedding_dim)
        self.pos_embedding_src = PositionalEncoding(embedding_dim, 33)
        self.pos_embedding_tgt = PositionalEncoding(embedding_dim, 73)
        self.transformer = nn.Transformer(d_model=embedding_dim, nhead=num_heads, num_encoder_layers=num_layers, num_decoder_layers=num_layers, dropout=dropout, batch_first=True)
        self.fc_out = nn.Linear(embedding_dim, vocab_size_tgt)

    def forward(self, src, tgt):
        src = self.embedding_src(src)
        tgt = self.embedding_tgt(tgt)
        batch_size = src.shape[0]
        pos_src = self.pos_embedding_src.get_positional_vector().to(device)
        pos_src = pos_src.unsqueeze(0).repeat(batch_size, 1, 1)
        pos_tgt = self.pos_embedding_tgt.get_positional_vector().to(device)
        pos_tgt = pos_tgt.unsqueeze(0).repeat(batch_size, 1, 1)
        src = src + pos_src
        tgt = tgt + pos_tgt
        output = self.transformer(src, tgt)
        output = self.fc_out(output)
        return output



# モデルのハイパーパラメータ
embedding_dim = 512
num_heads = 8
num_layers = 6
device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')
# モデルの初期化
# モデルの初期化
model = TransformerModel(vocab_size_src, vocab_size_tgt, embedding_dim, num_heads, num_layers, device, dropout=0).to(device)

# 保存された重みをロード
model.load_state_dict(torch.load("./model_weight.pth", map_location=device))

FileNotFoundError: [Errno 2] No such file or directory: './model_weight.pth'

In [9]:
# モデルのハイパーパラメータ
embedding_dim = 512
num_heads = 8
num_layers = 6
lr_rate = 1e-5

device = torch.device('cuda:7' if torch.cuda.is_available() else 'cpu')

# モデルの初期化
model = TransformerModel(vocab_size_src, vocab_size_tgt, embedding_dim, num_heads, num_layers, device).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=PADDING_ID)
optimizer = optim.Adam(model.parameters(), lr=lr_rate)

# トレーニングループ
num_epochs = 100

def train_epoch(model, dataloader, criterion, optimizer):
    model.train()
    epoch_loss = 0
    for src, tgt in dataloader:
        optimizer.zero_grad()
        src = src.to(device)
        tgt = tgt.to(device)
        output = model(src, tgt)
        # tgt = F.one_hot(tgt, tgt_vocab_size)
        output = output.permute(0, 2, 1) #なぜ(バッチサイズ、　シークエンス長、　vocab_size) => (バッチサイズ、　vocab_size、　シークエンス長)にする必要があるのか
        loss = criterion(output, tgt)
        loss.backward()##ここが問題　r
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss/len(dataloader)

for epoch in tqdm(range(num_epochs)):
    train_loss = train_epoch(model, train_dataloader, criterion, optimizer)
    if (epoch+1)%10 == 0:
        print(f'Epoch {epoch+1}, Train Loss: {train_loss:.4f}')

  0%|          | 0/100 [00:00<?, ?it/s]


TypeError: cannot unpack non-iterable NoneType object

In [5]:
torch.save(model.state_dict(),'model_weight.pth')